In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

In [ ]:
def create_link(ecli_from, ecli_to, sparql):
    base_query = '''
    prefix dcterm: <http://purl.org/dc/terms/>
    insert data 
    {{<http://deeplink.rechtspraak.nl/uitspraak?id={}>
        dcterm:references 
        <http://deeplink.rechtspraak.nl/uitspraak?id={}> 
    .}}'''
    query = base_query.format(ecli_from, ecli_to)
    sparql.method = 'POST'
    sparql.setQuery(query)
    res = sparql.query()

In [6]:
sparql = SPARQLWrapper("http://localhost:8889/blazegraph/namespace/hogeraad/sparql")

In [ ]:
#Create for Gijs' network

import pandas as pd
filepath_csv = '/media/sf_VBox_Shared/CaseLaw/graphs/test3 wg aanspr.csv'
links_wgaans = pd.read_csv(filepath_csv, sep=';', header=0)
links_wgaans.columns = ['id', 'reference']
links_wgaans = links_wgaans.drop_duplicates()

In [7]:
for i, r in links_wgaans.iterrows():
    create_link(r['id'], r['reference'], sparql)

HTTPError: HTTP Error 503: Service Unavailable

In [ ]:
# Create for previously found links
import sqlite3

conn = sqlite3.connect('rechtspraak.db')
c = conn.cursor()

In [ ]:
links = pd.DataFrame(c.execute('''select l.id, l.reference
            from uitspraken_links_ljn l join 
                ( select id from uitspraken_meta ) s
                on s.id = l.reference
        ''').fetchall(), columns=['id', 'reference'])

In [ ]:
conn.close()

In [ ]:
for i, r in links.iterrows():
    create_link(r['id'], r['reference'], sparql)

In [ ]:
#Make a query
queryString = "prefix dcterm: <http://purl.org/dc/terms/> select (COUNT(?s) as ?count) {?s dcterm:references ?o}"
sparql.setQuery(queryString)
sparql.setReturnFormat(JSON)
ret = sparql.query()
ret.convert()

In [ ]:
print(links_wgaans.shape)
print(links.shape)

In [ ]:
links.head()